# HOP-1: volume (patient_count, claim_count) in undirected version of the hop teaming, (A,B) and (B,A) should be the same

Description: volume of (A,B) and (B,A) in undirected version of the hop teaming should be the same. For example: 
* 1111(from npi), 2222(to npi), 45(patient count), 50(claim count) should be the same as 
* 2222(from npi), 1111(to npi), 45(patient count), 50(claim count). 

Starting Author: Amy Jin (amy@careset.com)

Date: July 20th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [5]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [6]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) bene_cnt:               patient count column 
# 4) clm_cnt:                claim count column
# 5) from_npi:               from npi column 
# 5) to_npi:                 to npi column 
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out:
#       - the total count of (from_npi, to_npi) pair fail the test and
#       - total count of (from_npi, to_npi) pair in the table.


def hop_1(db_name, table_name, bene_cnt, clm_cnt, from_npi, to_npi):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
                SELECT COUNT(*)
                FROM {db}.{t} AS t1
                JOIN {db}.{t} AS t2
                ON t1.{from_npi} = t2.{to_npi} AND t1.{to_npi} = t2.{from_npi}
                WHERE t1.{col1} <> t2.{col1} OR t1.{col2} <> t2.{col2};
        '''.format(db = db_name, t = table_name, col1 = bene_cnt, col2 = clm_cnt, from_npi = from_npi, to_npi = to_npi))

        cur.execute(query)
    
        for row in cur.fetchall():
            for i in range(0,len(row)):
                if row[i] == 0:
                    print ("Hop-1 test result: PASS" + '\n')
                else:
                    print ("Hop-1 test result: FAIL"  + '\n')
                    print ("The total count of ({}, {}) pair fail the test is:".format(from_npi, to_npi)  + '\n')  
                    print (str(row[i]))

                    query = ('''
                            SELECT COUNT(*)
                            FROM {db}.{t} AS t1;
                    '''.format(db = db_name, t = table_name, col1 = bene_cnt, col2 = clm_cnt, from_npi = from_npi, to_npi = to_npi))

                    cur.execute(query)

                    print ("The total count of (from_npi, to_npi) pair is:" + '\n')        
                    for row in cur.fetchall():
                        for i in range(0,len(row)):
                            print (str(row[i]))
                
        cur.close()
        connection.close()            

## Test Example

In [7]:
hop_1('_amy', 'test_hop_undirected_good', 'patient_count','transaction_count', 'npi', 'npi_DEST')

Hop-1 test result: PASS



In [8]:
hop_1('_amy', 'test_hop_undirected_bad', 'patient_count','transaction_count', 'npi', 'npi_DEST')

Hop-1 test result: FAIL

The total count of (npi, npi_DEST) pair fail the test is:

56
The total count of (from_npi, to_npi) pair is:

652
